**This code enables manual filtering of days and hours belonging to estimated HWs during 2022 (Notebook 1). Particularly, it calculates the average over the selected days and hours. The output is a single CSV file containing the average air temperature for each station.**

It uses the altitude-corrected dataset from Notebook 2.

In [1]:
import pandas as pd
import ipywidgets as widgets
from ipywidgets import interactive, Output

In [2]:
arpa = pd.read_parquet("./Data/CML-ARPA/ARPA/ARPA_2022_hourly_altcorr.parquet")
cml = pd.read_parquet("./Data/CML-ARPA/CML/CML_2022_hourly_altcorr.parquet")

In [3]:
arpa, cml

(                  Datetime  Temp     Id                         station  \
 0      2022-01-01 01:00:00 -0.55   2001                 Milano Lambrate   
 1      2022-01-01 02:00:00 -0.42   2001                 Milano Lambrate   
 2      2022-01-01 03:00:00  0.68   2001                 Milano Lambrate   
 3      2022-01-01 04:00:00  1.38   2001                 Milano Lambrate   
 4      2022-01-01 05:00:00  0.75   2001                 Milano Lambrate   
 ...                    ...   ...    ...                             ...   
 297835 2022-12-31 20:00:00  8.57  19426  Mariano Comense campo sportivo   
 297836 2022-12-31 21:00:00  8.35  19426  Mariano Comense campo sportivo   
 297837 2022-12-31 22:00:00  8.17  19426  Mariano Comense campo sportivo   
 297838 2022-12-31 23:00:00  8.05  19426  Mariano Comense campo sportivo   
 297839 2023-01-01 00:00:00  8.10  19426  Mariano Comense campo sportivo   
 
               lat      long urban/natural  altitude1 Altitude-group  \
 0       45.49

## **Merge ARPA and CML into one df**

In [4]:
arpa = arpa.drop(columns=['Id'])

In [5]:
arpa['source'] = 'arpa'
cml['source'] = 'cml'

In [6]:
merged_df = pd.concat([arpa, cml], axis=0, ignore_index=True)
print(merged_df.head())

             Datetime  Temp          station       lat      long  \
0 2022-01-01 01:00:00 -0.55  Milano Lambrate  45.49678  9.257515   
1 2022-01-01 02:00:00 -0.42  Milano Lambrate  45.49678  9.257515   
2 2022-01-01 03:00:00  0.68  Milano Lambrate  45.49678  9.257515   
3 2022-01-01 04:00:00  1.38  Milano Lambrate  45.49678  9.257515   
4 2022-01-01 05:00:00  0.75  Milano Lambrate  45.49678  9.257515   

  urban/natural  altitude1 Altitude-group  Temp_corrected source   Id  
0       natural      121.5        100-150           -0.42   arpa  NaN  
1       natural      121.5        100-150           -0.29   arpa  NaN  
2       natural      121.5        100-150            0.81   arpa  NaN  
3       natural      121.5        100-150            1.51   arpa  NaN  
4       natural      121.5        100-150            0.88   arpa  NaN  


In [7]:
num_unique_stations = merged_df['station'].nunique()
print(f"Number of unique stations: {num_unique_stations}")

Number of unique stations: 97


## **Filtering days and times**

Interactive selection of days and hours.

In [8]:
cml['Datetime'] = pd.to_datetime(cml['Datetime'])

# Extract date and time from 'Datetime'
merged_df['date_only'] = merged_df['Datetime'].dt.date
merged_df['time_only'] = merged_df['Datetime'].dt.time

# Get unique dates and times from the data
unique_dates = sorted(merged_df['date_only'].unique())
unique_times = sorted(merged_df['time_only'].unique())

# Create widgets for selecting days and times
day_dropdown = widgets.SelectMultiple(
    options=[('All', 'all')] + [(str(i), i) for i in unique_dates],
    value=[unique_dates[0]],  # default value
    description='Days:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
)

time_dropdown = widgets.SelectMultiple(
    options=[('All', 'all')] + [(str(i), i) for i in unique_times],
    value=[unique_times[0]],  # default value
    description='Times:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
)

# Create an Output widget to display results
output = Output()

# Store the filtered dataframe in a global variable so it can be accessed later
filtered_data = None

# Function to filter the DataFrame based on selected days and times
def filter_by_day_and_time(selected_days, selected_times):
    global filtered_data  # Declare filtered_data as global to store it for later use

    # Clear previous output before showing new results
    with output:
        output.clear_output()

        # Convert 'selected_days' and 'selected_times' to the appropriate format
        if 'all' in selected_days:
            selected_days = unique_dates  # Include all dates if 'all' is selected
        if 'all' in selected_times:
            selected_times = unique_times  # Include all times if 'all' is selected

        # Filter the DataFrame based on selected days and times
        filtered_data = merged_df[merged_df['date_only'].isin(selected_days) & merged_df['time_only'].isin(selected_times)]
        
        if filtered_data.empty:
            print("No data available for selected filters.")
        else:
            # Display the filtered DataFrame in the output
            print(filtered_data)

# Create an interactive output with the widgets
interactive_output = interactive(filter_by_day_and_time, selected_days=day_dropdown, selected_times=time_dropdown)

# Display the widgets and the output
display(day_dropdown, time_dropdown, output)

SelectMultiple(description='Days:', index=(1,), layout=Layout(width='50%'), options=(('All', 'all'), ('2022-01…

SelectMultiple(description='Times:', index=(1,), layout=Layout(width='50%'), options=(('All', 'all'), ('00:00:…

Output()

In [13]:
filtered_data

,Datetime,Temp,station,lat,long,urban/natural,altitude1,Altitude-group,Temp_corrected,source,Id,date_only,time_only
1181,2022-02-19 06:00:00,8.42,Milano Lambrate,45.496780,9.257515,natural,121.5,100-150,8.55,arpa,NaN,2022-02-19,06:00:00
1182,2022-02-19 07:00:00,8.33,Milano Lambrate,45.496780,9.257515,natural,121.5,100-150,8.46,arpa,NaN,2022-02-19,07:00:00
1205,2022-02-20 06:00:00,9.25,Milano Lambrate,45.496780,9.257515,natural,121.5,100-150,9.38,arpa,NaN,2022-02-20,06:00:00
1206,2022-02-20 07:00:00,7.78,Milano Lambrate,45.496780,9.257515,natural,121.5,100-150,7.91,arpa,NaN,2022-02-20,07:00:00
1229,2022-02-21 06:00:00,7.08,Milano Lambrate,45.496780,9.257515,natural,121.5,100-150,7.21,arpa,NaN,2022-02-21,06:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
842190,2022-02-21 07:00:00,6.69,zerbolo,45.227222,8.985000,urban,102.0,100-150,6.94,cml,zerbolo,2022-02-21,07:00:00
842213,2022-02-22 06:00:00,2.05,zerbolo,45.227222,8.985000,urban,102.0,100-150,2.30,cml,zerbolo,2022-02-22,06:00:00
842214,2022-02-22 07:00:00,1.40,zerbolo,45.227222,8.985000,urban,102.0,100-150,1.65,cml,zerbolo,2022-02-22,07:00:00
842237,2022-02-23 06:00:00,-0.17,zerbolo,45.227222,8.985000,urban,102.0,100-150,0.08,cml,zerbolo,2022-02-23,06:00:00


In [14]:
# Calculate the number of NaNs and the total number of entries for each station
nan_counts = filtered_data.groupby('station')['Temp_corrected'].apply(lambda x: x.isna().sum())
total_counts = filtered_data.groupby('station')['Temp_corrected'].count()

# Calculate the percentage of NaNs for each station
nan_percentage = (nan_counts / (nan_counts + total_counts)) * 100

# Calculate the average temperature with one decimal place
station_avg_temp = filtered_data.groupby('station').agg(
    avg_temp=('Temp_corrected', 'mean'),
    lat=('lat', 'first'),
    long=('long', 'first'),
    source=('source', 'first')
).reset_index()

# Round avg_temp to one decimal
station_avg_temp['avg_temp'] = station_avg_temp['avg_temp'].round(1)

# Add the NaN percentage information
station_avg_temp['nan_percentage'] = station_avg_temp['station'].map(nan_percentage)

# Remove stations with too many NaNs (more than 20%)
threshold = 21
station_avg_temp_filtered = station_avg_temp[station_avg_temp['nan_percentage'] <= threshold]

print(station_avg_temp.to_string())
print(station_avg_temp_filtered)

                              station  avg_temp        lat      long source  nan_percentage
0                        Arconate SMR       4.8  45.548521  8.847327   arpa             0.0
1               Bereguardo Fornasetta       4.4  45.262781  9.021911   arpa             0.0
2                  Bergamo v.Stezzano       6.3  45.660521  9.658768   arpa             0.0
3             Busto Arsizio v.Rossini       4.3  45.626390  8.823884   arpa             0.0
4                  Castello d'Agogna        3.9  45.246802  8.700569   arpa             0.0
5   Cavaria con Premezzo v. Cantalupa       3.2  45.689494  8.807690   arpa             0.0
6                     Cavenago d'Adda       6.1  45.269274  9.562660   arpa             0.0
7        Cinisello Balsamo Parco Nord       5.5  45.542665  9.205603   arpa             0.0
8                 Corsico v.le Italia       6.2  45.436113  9.097412   arpa             0.0
9                      Ferno v.Di Dio       3.2  45.619248  8.756977   arpa     

In [15]:
station_avg_temp_filtered

,station,avg_temp,lat,long,source,nan_percentage
0,Arconate SMR,4.8,45.548521,8.847327,arpa,0.0
1,Bereguardo Fornasetta,4.4,45.262781,9.021911,arpa,0.0
2,Bergamo v.Stezzano,6.3,45.660521,9.658768,arpa,0.0
3,Busto Arsizio v.Rossini,4.3,45.626390,8.823884,arpa,0.0
4,Castello d'Agogna,3.9,45.246802,8.700569,arpa,0.0
...,...,...,...,...,...,...
91,vigevano,5.3,45.294722,8.854444,cml,20.0
92,vignate,7.3,45.500278,9.380556,cml,0.0
93,villasanta,5.5,45.616944,9.301111,cml,0.0
95,vizzola,3.4,45.652500,8.684167,cml,0.0


In [12]:
# Export to csv
# First we drop nan column
station_avg_temp_filtered = station_avg_temp_filtered.drop(columns=['nan_percentage'])

# Here change the output based on the selected HW
station_avg_temp_filtered.to_csv('./Data/CML-ARPA/HW_extracted/CML-ARPA_HP_Feb_2022_MUHI.csv', index=False)